#Introduction
We will be using Google Colab to implement this example that covers linear, non-linear, and quantile regression. First, go to https://colab.research.google.com/. Click File -> New notebook in Drive, and then change the runtime to R (Runtime -> Change runtime type, then pick R in the dropdown). We will not be using GPUs, so keep the CPU box checked.

# Installation
To install Keras, run the following code. Colab already has Python and Tensorflow modules installed, so we do not need to do anything particularly complicated here.

In [ ]:
remotes::install_github("rstudio/tensorflow")

In [ ]:
install.packages("keras3")

In [ ]:
library("keras3")

Set seeds for reproducibility.

In [ ]:
tensorflow::set_random_seed(1)

# Extracting a dataset

We will be downloading a dataset of US wildfires, which is hosted on github. The data are described in [Richards and Huser (2022)](https://arxiv.org/abs/2208.07581) and also [here](https://github.com/Jbrich95/pinnEV/blob/main/R/USWild.R).

We take only the last 5 years of the dataset (there's seven months per year, so 7*5 = 35 observations).

In [ ]:
file_url <- "https://github.com/Jbrich95/pinnEV/blob/main/data/USWild.rda?raw=true"
load(url(file_url))
Y = USWild$BA[127:161, , ]
X = array(dim = c(35, 129, 61, 22))
X[, , , 1] = USWild$X$X.t2m[127:161, , , ]
X[, , , 2] = USWild$X$X.SPI[127:161, , , ]
X[, , , -c(1, 2)] = USWild$X$X.N[127:161, , , 1:20]

The data are arranged on a regular spatial grid, and we have 22 covariates. The first two covariates are monthly air temperature (t2m) are standardised preciptation index (SPI); the remaing covariates are a mixture of metereological and land cover descriptors.

In [ ]:
dim(X)
cov.names <- c("t2m", "SPI", USWild$X$cov.names[1:20])
print(cov.names) #Prints names of covariates

The response is monthly square-root burnt area due to wildfires in the US. Burnt area data are very heavy-tailed, so taking the square root is going to make modelling a bit more numerically stable.

In [ ]:
image(log(1 + Y[1, , ]), col = heat.colors(20, rev = T), asp = 0.625)

We can also look at some of the covariates. Note that these maps are for the first month in the dataset.

In [ ]:
par(mfrow = c(1, 2))
image(X[1, , , 1],
      col = heat.colors(20, rev = T),
      main = "temperature",
      asp = 0.625)
image(X[1, , , 2],
      col = terrain.colors(20, rev = F),
      main = "SPI",
      asp = 0.625)

We will model extremes of positive $Y$, i.e., square-root burnt aree. Note that I'm completely removing all zero values - we won't be needing these here - and now our data are stored vectors, rather than arrays.

In [ ]:
X.positive <- apply(X, 4, function(x) x[Y > 0])
Y.positive <- sqrt(Y[Y > 0])

When it comes to ML modelling, training is more numerically stable if your data are standardised/normalised. Here, we scale the covariates to have zero mean, unit variance.

In [ ]:
means <- apply(X.positive, 2, mean)
sds <- apply(X.positive, 2, sd)
X.scaled <- apply(as.matrix(1:ncol(X.positive)), 1, function(ind)
  (X.positive[, ind] - means[ind]) / sds[ind])

# Standard prediction

We now build and train a standard prediction model with Keras. To account for the heavy-tailedness of the response, we will target $\mathbb{E} \log Y$ using the mean-squared error loss function.

For simplicity, we will sample 45000 observations for training and 5000 for testing.

In [ ]:
n = length(Y.positive)
train_sample_idx <- sample(1:n, 45000)
test_sample_idx <- sample((1:n)[-train_sample_idx], 5000)
Y_train <- Y.positive[train_sample_idx]
X_train <- X.scaled[train_sample_idx, ]
Y_test <- Y.positive[test_sample_idx]
X_test <- X.scaled[test_sample_idx, ]

We first show how to build Keras models sequentially. Here we have two hidden layers, each of width 64, with ReLU activation functions. The final layer has a linear activation function:

In [ ]:
model <- keras_model_sequential()

model %>%

  # Adds a densely-connected layer with 64 units to the model:
  layer_dense(units = 64, activation = 'relu') %>%

  # Add another:
  layer_dense(units = 64, activation = 'relu') %>%


  # Add a final layer with 1 ouput
  layer_dense(units = 1, activation = 'linear')

Now, compile the model with a loss function and an optimizer. Here we use Adam with standard hyper-parameters, and the MSE loss function to do regular prediction. We also evaluated the mean absolute error, and track its value during training.

In [ ]:
model %>% compile(
  optimizer = "adam",
  loss = "mean_squared_error",
  metrics = list("mean_absolute_error")
)

Now fit the model. We train the model for 100 epochs, with an 80/20 validation data split. The default minibatch size is 16. Note that the model gradients are not evaluated on the validation data, and instead we can use the validation loss (i.e., the loss evaluated on the validation data) to motivate hyperparameters (e.g., neural net architecture) choices.

If you choose to cancel training (ctrl + C, or the big red stop button), then the current model state will be saved and accessible. Feel free to set verbose = 1 to track the training.

In [ ]:
history <- model %>% fit(
  x = as.matrix(X_train),
  y = as.matrix(log(Y_train)),
  epochs = 100,
  validation_split = 0.2,
  verbose = 1,
  shuffle = T
)

Plot the training history, and print the summary of the architecture.

In [ ]:
plot(history)
summary(model)

We can see the the model begins to overfit quickly. Let's re-run the model with a checkpointed training scheme; the model loss will be evaluated at each epoch, and will be saved if it provides the best fit on the validation data. In this way, we can re-load the model state/checkpoint which best generalised to unseen data (in this case, the validation data).

In [ ]:
model <- keras_model_sequential() %>%
  layer_dense(units = 64, activation = 'relu') %>%
  layer_dense(units = 64, activation = 'relu') %>%
  layer_dense(units = 1, activation = 'linear') %>%
  compile( optimizer = "adam", loss = "mean_squared_error",
           metrics = list("mean_absolute_error"))

checkpoint <- callback_model_checkpoint(
  paste0("checkpoints/LSE.weights.h5"),
  monitor = "val_loss",
  verbose = 0,
  save_best_only = TRUE,
  save_weights_only = TRUE,
  mode = "min",
  save_freq = "epoch"
)



In [ ]:
history <- model %>% fit( x = X_train,
                          y = as.matrix(log(Y_train)),
                          epochs=50,
                          batch_size=16,
                            verbose = 1,
                          callbacks = list(checkpoint),
                          validation_split=0.2)

In [ ]:
plot(history)

Now, load that best saved checkpoint:

In [ ]:
model <- load_model_weights(model,
                                 filepath = paste0("checkpoints/LSE.weights.h5"))

We now get the predictions from the fitted model, using the standard `predict` function.

In [ ]:
predictions <- model %>% predict(X_test)
plot(log(Y_test), predictions)
abline(a = 0, b = 1)

For comparsion, we can also fit a linear model. This can be built and trained in Keras as well; recall that a linear model is a special case of a one-layered MLP with one neuron and with a linear final activation layer.

In [ ]:
linear.model <- keras_model_sequential() %>%
  layer_dense(units = 1, activation = 'linear') %>%
  compile(
    optimizer = "adam",
    loss = "mean_squared_error",
    metrics = list("mean_absolute_error")
  )


history <- linear.model %>% fit(
  x = X_train,
  y = as.matrix(log(Y_train)),
  epochs = 15,
  verbose = 0,
  validation_split = 0.2
)

This model trains much faster than the MLP…

In [ ]:
plot(history)

Here we plot the out-of-sample predictions. As you would expect, the model performs much more poorly than the deep neural network model.

In [ ]:
linear_predictions <- linear.model %>% predict(X_test)

In [ ]:
plot(log(Y_test), linear_predictions, asp = 1)
abline(a = 0, b = 1)

We may also want to see how the predictions change with the covariates. Below, we plot maps of the predicted log(Y) for a specific month t. Note that we have to use the original covariates here, rather than just those conditional on Y>0; this means we also need to remember to scale our original covariate vector, in the same was as we did above for the subset of covariates used in the model.

In [ ]:
par(mfrow = c(1, 3))

t <- 1
X.plot <- X[t, , , ]
dim(X.plot) <- c(prod(dim(X.plot)[1:2]), dim(X.plot)[3])
# Apply scaling
X.plot.scaled <- apply(as.matrix(1:ncol(X.plot)), 1, function(ind)
  (X.plot[, ind] - means[ind]) / sds[ind])

linear.predictions <- linear.model %>% predict(X.plot.scaled)

dim(linear.predictions) <- dim(Y)[2:3]
linear.predictions[Y[t, , ] < 0] = NA
#linear.predictions[Y[t, , ] <= 0] = NA #Comment for US-wide prediction
image(log(Y[t, , ]), main = "Observation",asp=0.625)


image(linear.predictions, main = "Linear model predictions",asp=0.625)

NN.predictions <- model %>% predict(X.plot.scaled)

dim(NN.predictions) <- dim(Y)[2:3]
NN.predictions[Y[t, , ] < 0] = NA
#NN.predictions[Y[t, , ] <= 0] = NA #Comment for US-wide prediction
image(NN.predictions, main = "MLP model predictions",asp=0.625)

# Quantile regression

Now, we will perform quantile regression using deep neural networks. We will target the $\tau=0.8$
quantile of $Y|X$

by using the pinball loss function. We will supply this as a custom loss function to Keras.

All loss functions in Keras take two in puts: `y_true`, the true values of your response data, and `y_pred`, the predicted values that are outputted from your neural network model. Note that `y_pred` does not need to be directly connected to $Y$, e.g., the expectation or a quantile; below, `y_pred` will correspond to the GPD parameters.

Loss functions must be differentiable. To ensure this is the case, we build them using backend Keras functions. This can sometimes make it quite difficult to write custom loss functions in Keras; for these examples, the loss functions are quite simple, but you may need to get a bit creative if your loss function uses non-standard operations!

In [ ]:
tau <- 0.8 # Set quantile level
tilted_loss <- function(y_true, y_pred) {
  #K <- backend()

  error = y_true - y_pred
  return(op_mean(op_maximum(tau * error, (tau - 1) * error)))
}

Now, we use the same hidden layer model, but target the 80% quantile of Y.

In [ ]:
u.model <- keras_model_sequential() %>%
  layer_dense(units = 64, activation = 'relu') %>%
  layer_dense(units = 64, activation = 'relu')

This time, I specify an exponential activation in the final layer, to ensure that the quantile is strictly positive; recall that we removed all non-positive values of the response.

In [ ]:
u.model <- u.model %>% layer_dense(
  units = 1,
  activation = "exponential")

We compile and fit the model. This time, I supply a larger minibatch size, in order to speed-up training slightly.

In [ ]:
u.model %>%
  compile(optimizer = "adam", loss = tilted_loss)

checkpoint <- callback_model_checkpoint(
  paste0("checkpoints/u.weights.h5"),
  monitor = "val_loss",
  verbose = 0,
  save_best_only = TRUE,
  save_weights_only = TRUE,
  mode = "min",
  save_freq = "epoch"
)

history <- u.model %>% fit(
  x = X_train,
  y = as.matrix(Y_train),
  batch_size = 64,
  epochs = 40,
    verbose = 0,
  callbacks = list(checkpoint),
  validation_split = 0.2
)

In [ ]:
plot(history)

Now load that best checkpoint:

In [ ]:
u.model <- load_model_weights(u.model,
                                 filepath = paste0("checkpoints/u.weights.h5"))

Evaluate the test predictions of the $\tau$-quantile. As a sanity check, we will evalute the proportion of test data that exceed the predicted quantile; this should be close to $1-\tau$.

In [ ]:
test.pred.theshold <- u.model %>% predict(X_test)
mean(Y_test < test.pred.theshold) # Should be close to tau

We now plot maps of the estimated $\tau$
-quantile for a specified month. Below, we take the third month, but feel free to change `t` in the block of code.

In [ ]:
par(mfrow = c(1, 2))

t <- 3
X.plot <- X[t, , , ]
dim(X.plot) <- c(prod(dim(X.plot)[1:2]), dim(X.plot)[3])
# Apply scaling
X.plot.scaled <- apply(as.matrix(1:ncol(X.plot)), 1, function(ind)
  (X.plot[, ind] - means[ind]) / sds[ind])

image(log(Y[t, , ]), main = "Observation", asp = 0.625)

NN.predictions <- u.model %>% predict(X.plot.scaled)

dim(NN.predictions) <- dim(Y)[2:3]
NN.predictions[Y[t, , ] < 0] = NA
#NN.predictions[Y[t, , ] <= 0] = NA #Comment to provide US-wide predictions.
image(log(NN.predictions), main = "Threshold prediction", asp = 0.625)
